In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline                                           
from sklearn.preprocessing import StandardScaler                                     
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import roc_auc_score as AUC
from sklearn.base import clone
from sklearn import tree
from sklearn.model_selection import ParameterGrid, GridSearchCV
from joblib import Parallel, delayed
from multiprocessing import Pool
from tqdm import tqdm
from scipy.spatial import distance
from skbio.stats.ordination import pcoa
import umap
from sklearn.metrics import make_scorer
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

# Get data

In [3]:
# read meta data
df_meta = pd.read_csv('../../our_data/meta_data.csv', index_col=0)
df_meta = df_meta[df_meta.Diet=='Inulin']
df_meta = df_meta[df_meta.Day != 0] # remove day 0

# read SCFA data
df_scfa = pd.read_csv('../../our_data/SCFA.csv', index_col=0)

# read bacterial abundance (species level)
df_bac = pd.read_csv('../../our_data/16S_absolute_abundance_species.csv', index_col=0)

# find common samples
common_samples = list(set(df_meta.index).intersection(df_scfa.index).intersection(df_bac.index))
df_meta = df_meta.loc[common_samples]
df_scfa = df_scfa.loc[common_samples]
df_bac = df_bac.loc[common_samples]

# remove species that are constant across all samples
df_bac = df_bac[list(df_bac.std()[df_bac.std()>0].index)]
df_bac = df_bac[['Bacteroides-acidifaciens','Muribaculaceae','Faecalibaculum','Parasutterella','Bacteroides']]

# Run ElasticNet

## intrapolation

In [6]:
results = []
for k,group_to_exclude in enumerate(['A','B','C','D']):

    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.RandomizedGroup!=group_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.RandomizedGroup==group_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))

    # get weights of training sets
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression with weights
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])
        param_grid = {
        #'selectfrommodel__estimator__alpha':[10**v for v in [-4,-3,-2,-1,0]], # too large alpha will produce a null model (all features are 0)
        'randomforestregressor__max_features':['auto','sqrt','log2',0.16,0.32,0.64],
        'randomforestregressor__max_depth':[2,4,8,16],
        'randomforestregressor__min_samples_split':[2,4,8,16],
        'randomforestregressor__min_samples_leaf':[1,2,4]
        }

        clf = RandomForestRegressor(n_estimators=200,random_state=0,oob_score=True)
        pipe = make_pipeline(StandardScaler(), clf)  
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Intrapolation, group %s, %s, best score and parameter combination = '%(group_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')

        # predict training set
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)

        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['intrapolation', scfa, group_to_exclude, 'test', sample_, day_, obs_, pred_])

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 16.0min finished


Intrapolation, group A, Acetate, best score and parameter combination = 
0.018505759683547264
{'randomforestregressor__max_depth': 16, 'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 15.7min finished


Intrapolation, group A, Propionate, best score and parameter combination = 
0.3150799043698179
{'randomforestregressor__max_depth': 4, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


...

Intrapolation, group D, Acetate, best score and parameter combination = 
0.05637322247340748
{'randomforestregressor__max_depth': 2, 'randomforestregressor__max_features': 0.64, 'randomforestregressor__min_samples_leaf': 4, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 10.7min finished


Intrapolation, group D, Propionate, best score and parameter combination = 
0.3899731390264538
{'randomforestregressor__max_depth': 8, 'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  6.6min finished


Intrapolation, group D, Butyrate, best score and parameter combination = 
0.34454170626710157
{'randomforestregressor__max_depth': 8, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_samples_leaf': 4, 'randomforestregressor__min_samples_split': 16}




## extrapolation

In [7]:
for k,vendor_to_exclude in enumerate(['Beijing','Guangdong','Hunan','Shanghai']):
        
    # split train/test data
    mice_to_keep = list(set(df_meta[df_meta.Vendor!=vendor_to_exclude].MiceID))
    samples_to_keep = list(set(df_meta[df_meta.MiceID.isin(mice_to_keep)].index))
    mice_to_exclude = list(set(df_meta[df_meta.Vendor==vendor_to_exclude].MiceID))
    samples_to_exclude = list(set(df_meta[df_meta.MiceID.isin(mice_to_exclude)].index))

    # get weights of training sets
    xdata_train = np.asarray(df_bac.loc[samples_to_keep].values)
    xdata_test = np.asarray(df_bac.loc[samples_to_exclude].values)
    
    # run random forest regression with weights
    for scfa in ['Acetate','Propionate','Butyrate']:                
        ydata_train = np.asarray(df_scfa.loc[samples_to_keep, scfa])
        ydata_test = np.asarray(df_scfa.loc[samples_to_exclude, scfa])
        param_grid = {
        #'selectfrommodel__estimator__alpha':[10**v for v in [-4,-3,-2,-1,0]], # too large alpha will produce a null model (all features are 0)
        'randomforestregressor__max_features':['auto','sqrt','log2',0.16,0.32,0.64],
        'randomforestregressor__max_depth':[2,4,8,16],
        'randomforestregressor__min_samples_split':[2,4,8,16],
        'randomforestregressor__min_samples_leaf':[1,2,4]
        }

        clf = RandomForestRegressor(n_estimators=200,random_state=0,oob_score=True)
        pipe = make_pipeline(StandardScaler(), clf)  
        CV = GridSearchCV(pipe, param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=2)
        CV.fit(xdata_train, ydata_train)

        print('Extrapolation, vendor %s, %s, best score and parameter combination = '%(vendor_to_exclude, scfa))
        print(CV.best_score_)    
        print(CV.best_params_)    
        print('\n')   

        # predict training set
        ydata_train_predicted = CV.predict(xdata_train)
        ydata_test_predicted = CV.predict(xdata_test)

        for sample_, obs_, pred_ in zip(samples_to_keep, ydata_train, ydata_train_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'train', sample_, day_, obs_, pred_])
        for sample_, obs_, pred_ in zip(samples_to_exclude, ydata_test, ydata_test_predicted):
            day_ = df_meta.loc[sample_,'Day']
            results.append(['extrapolation', scfa, vendor_to_exclude, 'test', sample_, day_, obs_, pred_])


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   57.0s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  3.6min finished


Extrapolation, vendor Beijing, Acetate, best score and parameter combination = 
0.12205265510108129
{'randomforestregressor__max_depth': 8, 'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  3.0min finished


Extrapolation, vendor Beijing, Propionate, best score and parameter combination = 
0.4991474100399843
{'randomforestregressor__max_depth': 8, 'randomforestregressor__max_features': 0.64, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 4}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


...

Extrapolation, vendor Shanghai, Acetate, best score and parameter combination = 
-0.07851546896005376
{'randomforestregressor__max_depth': 16, 'randomforestregressor__max_features': 0.16, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  2.9min finished


Extrapolation, vendor Shanghai, Propionate, best score and parameter combination = 
0.09757963636471542
{'randomforestregressor__max_depth': 2, 'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__min_samples_leaf': 4, 'randomforestregressor__min_samples_split': 16}


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  4.1min finished


Extrapolation, vendor Shanghai, Butyrate, best score and parameter combination = 
-0.04953320214094832
{'randomforestregressor__max_depth': 2, 'randomforestregressor__max_features': 'sqrt', 'randomforestregressor__min_samples_leaf': 4, 'randomforestregressor__min_samples_split': 16}




## save results to file

In [8]:
df_prediction = pd.DataFrame(results, columns=['PerturbationType','SCFA','Permutation','PredictionType','SampleID','Day','ObservedValue','PredictedValue'])
df_prediction.to_csv('rf_prediction_predictor_speciesonly_rf_degraders.csv')